In [ ]:
import os
import sys
import numpy as np
import numpy.matlib as npm
import utm
import ast
import copy
import math
import scipy
import pandas as pd
from scipy.stats import norm
from scipy.stats import distributions
from operator import itemgetter
from numba import jit
import cartopy
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import ticker

In [ ]:
def plot_pois(pois_coords, ind, event_dict):

    ev_lon = event_dict['lon']
    ev_lat = event_dict['lat']
    
    proj = cartopy.crs.PlateCarree()
    fig = plt.figure(figsize=(16, 8))

    ax = plt.axes(projection=cartopy.crs.Mercator())

    # coastline = cartopy.feature.GSHHSFeature(scale='low', levels=[1])
    coastline = cartopy.feature.GSHHSFeature(scale='high', levels=[1])
    ax.add_feature(coastline, edgecolor='#000000', facecolor='#cccccc', linewidth=1)
    ax.add_feature(cartopy.feature.BORDERS.with_scale('50m'))
    ax.add_feature(cartopy.feature.STATES.with_scale('50m'))
    ax.add_feature(cartopy.feature.OCEAN.with_scale('50m'))
    gl = ax.gridlines(crs=proj, draw_labels=True, linewidth=1,
                        color="#ffffff", alpha=0.5, linestyle='-')
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True
    gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
    gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}

    ax.plot(pois_coords[ind,0], pois_coords[ind,1], markerfacecolor="#0ba518", marker="o",
                linewidth=0, markeredgecolor="#000000",
                transform=proj)#, zorder=10)

    ax.plot(ev_lon, ev_lat, linewidth=0, marker='*', markersize=10,
            markerfacecolor='red', markeredgecolor='#000000',
            transform=proj)
    
    ax.set_extent([np.min(pois_coords[:,0]), np.max(pois_coords[:,0]), np.min(pois_coords[:,1]), np.max(pois_coords[:,1])], crs=proj)

    ax.set_xlabel(r'Longitude ($^\circ$)', fontsize=14)
    ax.set_ylabel(r'Latitude ($^\circ$)', fontsize=14)

def plot_barycenters(bs_coords, prob, event_dict):

    # fig,axs = plt.subplots(2,1,figsize=(20,20),subplot_kw={'projection': cartopy.crs.Mercator()})
    # fig.set_tight_layout(True)
    ev_lon = event_dict['lon']
    ev_lat = event_dict['lat']

    fig = plt.figure(figsize=(16, 8))
    proj = cartopy.crs.PlateCarree()
 
    ax=plt.axes(projection=cartopy.crs.Mercator())
    coastline = cartopy.feature.GSHHSFeature(scale='high', levels=[1])
    ax.add_feature(coastline, edgecolor='#000000', facecolor='#cccccc', linewidth=1)
    ax.add_feature(cartopy.feature.BORDERS.with_scale('50m'))
    ax.add_feature(cartopy.feature.STATES.with_scale('50m'))
    ax.add_feature(cartopy.feature.OCEAN.with_scale('50m'))
    gl = ax.gridlines(crs=proj, draw_labels=True, linewidth=1,
                        color="#ffffff", alpha=0.5, linestyle='-')
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True
    gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
    gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
 
    lonlat_list=[list(x) for x in set(tuple(x) for x in bs_coords)]
    nloc=len(lonlat_list)
 
    sumprob=[]
    for iloc in range(nloc):
        locxy=lonlat_list[iloc]
        isel = [i for i, x in enumerate(list(bs_coords)) if np.allclose(x, locxy)]
        # isel = [i for i, x in enumerate(list(bs_coords)) if x == locxy]
        #print(len(isel))
        tmplist=[prob[i] for i in isel]
        sumprob.append(sum(tmplist))
    lon = [el[0] for el in lonlat_list]
    lat = [el[1] for el in lonlat_list]
 
    # lon = bs_coords[:,0] 
    # lat = bs_coords[:,1]
    cmap = ax.scatter(lon,lat,s=30, transform=proj,edgecolors='k',c=sumprob,cmap=plt.cm.plasma)#, label=labels[ic], c=colors[ic])
    cbar = plt.colorbar(cmap,ax=ax,extend='both',extendfrac='auto',aspect=30,format='%.0e',pad=0.02,
                 label='Cumulated Probability (for each barycenter)',shrink=0.8)
 
    ax.plot(ev_lon, ev_lat, linewidth=0, marker='*', markersize=10,
            markerfacecolor='red', markeredgecolor='#000000',
            transform=proj)
     
    ax.set_extent([np.min(lon)-0.2, np.max(lon)+0.2, np.min(lat)-0.2, np.max(lat)+0.2], crs=proj)

    cbar.locator = ticker.MaxNLocator(nbins=10)
    cbar.update_ticks()


In [3]:
# These functions are copied from the ptf_scaling_laws.py file
def scalinglaw_WC(**kwargs):
    '''
    Scaling law from Wells&Coppersmith (1994)
    '''

    mag        = kwargs.get('mag', None)
    type_scala = kwargs.get('type_scala', None)

    if (type_scala == 'M2L'):
        a =-2.440
        b =0.590
        y = 10.**(a+b*mag)

    elif (type_scala == 'M2W'):
        a=-1.010
        b=0.320
        y = 10.**(a+b*mag)

    else:
        raise Exception("Scaling law in scalinglaw_WC not recognized. Exit!")
        #print("Scaling law in scalinglaw_WC not recognized. Exit!")
        #sys.exit()

    return y

def scalinglaw_Murotani(**kwargs):
    '''
    Scaling law from Murotani et al (2013)
    '''

    mag        = kwargs.get('mag', None)
    type_scala = kwargs.get('type_scala', None)

    a    = -3.806
    b    = 1.000
    Area = 10**(a+b*mag)

    if (type_scala == 'M2L'):
        y = math.sqrt(2.5*Area)/2.5

    elif (type_scala == 'M2W'):
        y = math.sqrt(2.5*Area)

    else:
        raise Exception("Scaling law in scalinglaw_Murotani not recognized. Exit!")
        #print("Scaling law in scalinglaw_Murotani not recognized. Exit!")
        #sys.exit()

    return y

def mag_to_l_BS(**kwargs):

    mag = kwargs.get('mag', None)
    out = 1000.0 * scalinglaw_WC(mag=mag, type_scala='M2L')

    return out

def mag_to_w_BS(**kwargs):

    mag = kwargs.get('mag', None)
    out = 1000.0 * scalinglaw_WC(mag=mag, type_scala='M2W')

    return out

def mag_to_l_PS(**kwargs):

    mag = kwargs.get('mag', None)
    out = 1000.0 * scalinglaw_Murotani(mag=mag, type_scala='M2W')

    return out

def correct_BS_horizontal_position(**kwargs):

    mag = kwargs.get('mag', None)
    out = 0.5 * mag_to_l_BS(mag=mag)

    return out

def correct_PS_horizontal_position(**kwargs):

    mag = kwargs.get('mag', None)
    out = 0.5 * mag_to_l_PS(mag=mag)

    return out

def correct_BS_vertical_position(**kwargs):

    mag = kwargs.get('mag', None)
    out = math.sin(math.pi/4)*0.5 * mag_to_w_BS(mag=mag)

    return out

In [4]:
# These functions are copied from the ptf_load_event.py file
def int_quake_cat2dict(json_object):

    d = dict()

    # Event Ids
    try:
        origin_id =  str(json_object['features'][0]['properties']['originid'])
    except:
        origin_id =  str(json_object['features'][0]['properties']['originId'])

    event_id      =  str(json_object['features'][0]['properties']['eventId'])
    # author        =  str(json_object['features'][0]['properties']['author'])
    # version       =  str(json_object['features'][0]['properties']['version'])

    # Epicenter informations
    lon           =  float(json_object['features'][0]['geometry']['coordinates'][0])
    lat           =  float(json_object['features'][0]['geometry']['coordinates'][1])
    depth         =  float(json_object['features'][0]['geometry']['coordinates'][2])
    OT            =  str(json_object['features'][0]['properties']['time'])
    ev_type       =  str(json_object['features'][0]['properties']['type'])
    mag_type      =  str(json_object['features'][0]['properties']['magType'])
    place          =  str(json_object['features'][0]['properties']['place'])

    #utm conversion
    ee_utm = utm.from_latlon(lat, lon)

    # Specific Mag percentiles and covariant cov_matrix
    mag_percentiles = json_object['features'][0]['properties']['mag_percentiles']
    cov_matrix      = json_object['features'][0]['properties']['cov_matrix']
    
    pos_Sigma = cov_matrix.copy() #json_string['features'][0]['properties']['cov_matrix']

    cov_matrix['XX'] = float(cov_matrix['XX'])
    cov_matrix['XY'] = float(cov_matrix['XY'])
    cov_matrix['XZ'] = float(cov_matrix['XZ'])
    cov_matrix['YY'] = float(cov_matrix['YY'])
    cov_matrix['YZ'] = float(cov_matrix['YZ'])
    cov_matrix['ZZ'] = float(cov_matrix['ZZ'])

    pos_Sigma['XX']  = float(pos_Sigma['XX']) * 1e6
    pos_Sigma['XY']  = float(pos_Sigma['XY']) * 1e6
    pos_Sigma['XZ']  = float(pos_Sigma['XZ']) * 1e6
    pos_Sigma['YY']  = float(pos_Sigma['YY']) * 1e6
    pos_Sigma['YZ']  = float(pos_Sigma['YZ']) * 1e6
    pos_Sigma['ZZ']  = float(pos_Sigma['ZZ']) * 1e6

    mag_percentiles['p16']  = float(mag_percentiles['p16'])
    mag_percentiles['p50']  = float(mag_percentiles['p50'])
    mag_percentiles['p84']  = float(mag_percentiles['p84'])
    mag_sigma = 0.5 * (mag_percentiles['p84'] - mag_percentiles['p16'])

    d['eventid']        = event_id
    d['originid']       = origin_id
    d['lat']            = lat
    d['lon']            = lon
    d['depth']          = depth
    d['ot']             = OT
    d['mag']            = mag_percentiles['p50']
    d['mag_percentiles'] = mag_percentiles
    d['MagSigma']        = mag_sigma
    d['type']           = ev_type
    d['mag_type']       = mag_type
    d['ee_utm']         = ee_utm
    d['place']          = place

    # d['version']        = "%03d" % (float(version))
    # d['author']         = author
    # d['area']           = area
    # d['area_geo']       = area_geo
    # d['mag']            = mag
    # d['mag_values']     = mag_values
    # d['mag_counts']     = mag_counts
    # d['ct']             = creation_time
    # d['ot_year']        = origin_year
    # d['ot_month']       = origin_month
    # d['ot_day']         = origin_day

    d['cov_matrix']      = cov_matrix
    d['pos_Sigma']       = pos_Sigma

    d['ee_PosCovMat_2d'] = np.array([[cov_matrix['XX'], cov_matrix['XY']], \
                                     [cov_matrix['XY'], cov_matrix['YY']]])
    d['PosMean_2d']      = np.array([d['ee_utm'][0], \
                                     d['ee_utm'][1]])
    d['PosCovMat_3d']    = np.array([[cov_matrix['XX'], cov_matrix['XY'], cov_matrix['XZ']], \
                                     [cov_matrix['XY'], cov_matrix['YY'], cov_matrix['YZ']], \
                                     [cov_matrix['XZ'], cov_matrix['YZ'], cov_matrix['ZZ']]])
    d['PosCovMat_3dm']    = d['PosCovMat_3d']*1000000
    d['PosMean_3d']      = np.array([d['ee_utm'][0], \
                                     d['ee_utm'][1], \
                                     d['depth'] * 1000.0])

    # d['root_name']       = str(d['ot_year']) + str(d['ot_month']) + str(d['ot_day']) + '_' + \
    #                        d['area']
    #                        #str(d['eventid']) + '_' + str(d['version']) + '_' + d['area']

    return d

def compute_position_sigma_lat_lon(event_parameters):
    """
    REFERENCE LAT = YY
    REFERENCE LON = XX
    """

    bs_mag_max          = 8.1

    sigma               = event_parameters['sigma']
    event_mag           = event_parameters['mag_percentiles']['p50']
    event_mag_max       = event_parameters['mag_percentiles']['p50'] + \
                          event_parameters['MagSigma'] * sigma
    event_mag_sigma     = event_parameters['MagSigma']

    event_cov_xx        = event_parameters['pos_Sigma']['XX']
    event_cov_xy        = event_parameters['pos_Sigma']['XY']
    event_cov_yy        = event_parameters['pos_Sigma']['YY']


    mag_to_correct      = min(bs_mag_max, event_mag_max)

    delta_position_BS_h = correct_BS_horizontal_position(mag=mag_to_correct)
    delta_position_PS_h = correct_PS_horizontal_position(mag=event_mag + sigma * event_mag_sigma)

    position_BS_sigma_yy  = math.sqrt(abs(event_cov_yy)) + delta_position_BS_h
    position_BS_sigma_xx  = math.sqrt(abs(event_cov_xx)) + delta_position_BS_h
    position_BS_sigma_xy  = math.sqrt(abs(event_cov_xy)) + delta_position_BS_h

    event_parameters['position_BS_sigma_yy'] = position_BS_sigma_yy
    event_parameters['position_BS_sigma_xx'] = position_BS_sigma_xx
    event_parameters['position_BS_sigma_xy'] = position_BS_sigma_xy

    position_PS_sigma_yy  = math.sqrt(abs(event_cov_yy)) + delta_position_PS_h
    position_PS_sigma_xx  = math.sqrt(abs(event_cov_xx)) + delta_position_PS_h
    position_PS_sigma_xy  = math.sqrt(abs(event_cov_xy)) + delta_position_PS_h

    event_parameters['position_PS_sigma_yy'] = position_PS_sigma_yy
    event_parameters['position_PS_sigma_xx'] = position_PS_sigma_xx
    event_parameters['position_PS_sigma_xy'] = position_PS_sigma_xy

    return event_parameters


In [5]:
# These functions are copied from the ptf_ellipsoid.py file
def build_ellipsoid_objects(event_parameters, sigma_inn, sigma_out):

    ellipse = dict()

    location_ellipse_2d_BS_inn = build_location_ellipsoid_objects(event= event_parameters,
                                                                  sigma = sigma_inn,
                                                                  seismicity_type = 'BS')
    ellipse['location_ellipse_2d_BS_inn'] = location_ellipse_2d_BS_inn

    location_ellipse_2d_BS_out = build_location_ellipsoid_objects(event= event_parameters,
                                                                  sigma = sigma_out,
                                                                  seismicity_type = 'BS')
    ellipse['location_ellipse_2d_BS_out'] = location_ellipse_2d_BS_out

    location_ellipse_2d_PS_inn = build_location_ellipsoid_objects(event= event_parameters,
                                                                  sigma = sigma_inn,
                                                                  seismicity_type = 'PS')
    ellipse['location_ellipse_2d_PS_inn'] = location_ellipse_2d_PS_inn

    location_ellipse_2d_PS_out = build_location_ellipsoid_objects(event= event_parameters,
                                                                  sigma = sigma_out,
                                                                  seismicity_type = 'PS')
    ellipse['location_ellipse_2d_PS_out'] = location_ellipse_2d_PS_out

    return ellipse

def build_location_ellipsoid_objects(**kwargs):
    """
    From ellipsedata.m
    % Copyright (c) 2014, Hugo Gabriel Eyherabide, Department of Mathematics
    % and Statistics, Department of Computer Science and Helsinki Institute
    % for Information Technology, University of Helsinki, Finland.
    % All rights reserved.

    !!!! Difference with the original matlab function !!!!
    sigma in this python function is a float
    sigma in matlab is a vector

    """

    ee                 = kwargs.get('event', 'None')
    sigma              = kwargs.get('sigma', 'None')
    seismicity_type    = kwargs.get('seismicity_type', 'None')
   
    # Number of points to set the 2d ellipse
    nr_points = 1000
    
    sigma = float(sigma)

    # 2d Covariant matrix, eigenvalues and eignevectors
    if(seismicity_type == 'BS'):
       cov_matrix   = np.array([ [ee['position_BS_sigma_yy']**2, 0], [0, ee['position_BS_sigma_xx']**2] ])
    elif(seismicity_type == 'PS'):
       cov_matrix   = np.array([ [ee['position_PS_sigma_yy']**2, 0], [0, ee['position_PS_sigma_xx']**2] ])
    else:
       raise Exception('No seismicity type found. Exit')
       #sys.exit('No seismicity type found. Exit')

    # Center of the ellipse
    center = (ee['ee_utm'][1],ee['ee_utm'][0])

    PV, PD = np.linalg.eigh(cov_matrix)
    PV = np.sqrt(np.diag(PV))

    # Build points of ellipse
    theta = np.linspace(0,2*np.pi,nr_points)
    elpt  = np.dot(np.transpose(np.array([np.cos(theta), np.sin(theta)])) , PV)
    elpt  = np.dot(elpt, np.transpose(PD))

    # Add uncertainty
    elpt = elpt * sigma

    # shift to the center
    elpt    = np.transpose(elpt)
    elpt[0] = elpt[0] + center[0]
    elpt[1] = elpt[1] + center[1]
    elpt    = np.transpose(elpt)

    return elpt

In [ ]:
# These functions are copied from the ptf_pre_load.py file

def load_intensity_thresholds(data_folder):
    """
    READ THRESHOLDS
    """
    
    intensity_thresholds = os.path.join(data_folder, 'intensity_thresholds.npy')

    ith = np.load(intensity_thresholds, allow_pickle=True)
    intensity_measure = ith.item().keys()
    thresholds = ith.item()[list(intensity_measure)[0]]

    return thresholds, intensity_measure


def load_Model_Weights(data_folder):

    ps_type = 1

    weight_npy  = os.path.join(data_folder, 'ModelWeights.npy')
    # empty_space = "%10s" % ('')

    # print('Loading model weights dictionary:      <------ {}'.format(weight_npy))
    foe = np.load(weight_npy, allow_pickle=True)
    modelweights = foe.item()
    # for k in modelweights.keys():
    #     print('{} {}'.format(empty_space, k))

    # select only one type of ps_weigth
    # Only one weigth mode can be selected
    selected_index = np.where(modelweights['BS1_Mag']['Type'] == int(ps_type))
    modelweights['BS1_Mag']['Type'] = modelweights['BS1_Mag']['Type'][selected_index]
    modelweights['BS1_Mag']['Wei'] = modelweights['BS1_Mag']['Wei'][selected_index]

    selected_index = np.where(modelweights['PS2_Bar']['Type'] == int(ps_type))
    modelweights['PS2_Bar']['Type'] = modelweights['PS2_Bar']['Type'][selected_index]
    modelweights['PS2_Bar']['Wei'] = modelweights['PS2_Bar']['Wei'][selected_index]

    return modelweights


def load_region_files(Npoly, Ttype, data_folder):

    focal_mechanism_root_name    = 'MeanProb_BS4_FocMech_Reg'
    pyptf_focal_mechanism_dir    = os.path.join(data_folder, 'FocMech_PreProc')
    region_to_ignore             = [42]
    # region_to_ignore             = list(map(int, region_to_ignore))
    region_list                  = [x for x in range(Npoly) if x not in region_to_ignore]
    
    ModelsProb_Region_files      = dict()

    ## Define Region with PS now in this DICTIONARY
    region_ps_1                  = [3,24,44,48,49]
    region_ps_2                  = [10,16,54]
    region_ps_3                  = [27,33,35,36]
    region_listPs                =  [-1 for x in range(Npoly) if x not in region_to_ignore]
    for i in region_ps_1:
        region_listPs[i-1] = 1
    for i in region_ps_2:
        region_listPs[i-1] = 2
    for i in region_ps_3:
        region_listPs[i-1] = 3
    ModelsProb_Region_files['region_listPs'] = region_listPs

    regions_without_bs_focal_mechanism = []
    regions_with_bs_focal_mechanism    = []

    for iReg in range(len(region_list)+1):
        if (sum(itype for itype in Ttype[iReg] if itype == 1) > 0):
            regions_with_bs_focal_mechanism.append(iReg)
        else:
            regions_without_bs_focal_mechanism.append(iReg)

    ModelsProb_Region_files['ModelsProb_Region_files'] = []

    # print("Loading MeanProb_BS4_FocMech dict: <------ {}".format(focal_mechanism_root_name))
    for iReg in regions_with_bs_focal_mechanism:

        # define files
        filename  = focal_mechanism_root_name + '{}'.format(str(iReg+1).zfill(3)) + '.npy'
        f_FocMech = os.path.join(pyptf_focal_mechanism_dir,filename)
        ModelsProb_Region_files['ModelsProb_Region_files'].append(f_FocMech)

    for iReg in regions_without_bs_focal_mechanism:
        filename  = focal_mechanism_root_name + '{}'.format(str(iReg+1).zfill(3)) + '.npy'
        f_FocMech = os.path.join(pyptf_focal_mechanism_dir,filename)

        ModelsProb_Region_files['ModelsProb_Region_files'].append(f_FocMech)

    ModelsProb_Region_files['ModelsProb_Region_files'].sort()
    ModelsProb_Region_files['regions_with_bs_focal_mechanism']    = regions_with_bs_focal_mechanism
    ModelsProb_Region_files['regions_without_bs_focal_mechanism'] = regions_without_bs_focal_mechanism

    return ModelsProb_Region_files


In [7]:
# These functions are copied from the ptf_mix_utilities.py file
def conversion_to_utm(long, ee, PSBa):

    a = utm.from_latlon(np.array(long['Discretizations']['BS-2_Position']['Val_y']), np.array(long['Discretizations']['BS-2_Position']['Val_x']), ee['ee_utm'][2])
    long['Discretizations']['BS-2_Position']['utm_y']   = a[1]
    long['Discretizations']['BS-2_Position']['utm_x']   = a[0]
    long['Discretizations']['BS-2_Position']['utm_nr']  = a[2]
    long['Discretizations']['BS-2_Position']['utm_reg'] = a[3]

    a = utm.from_latlon(np.array(long['Discretizations']['PS-2_PositionArea']['Val_y']), np.array(long['Discretizations']['PS-2_PositionArea']['Val_x']), ee['ee_utm'][2])
    long['Discretizations']['PS-2_PositionArea']['utm_y']   = a[1]
    long['Discretizations']['PS-2_PositionArea']['utm_x']   = a[0]
    long['Discretizations']['PS-2_PositionArea']['utm_nr']  = a[2]
    long['Discretizations']['PS-2_PositionArea']['utm_reg'] = a[3]

    for i in range(len(PSBa['BarPSperModel'])):
        for j in range(len(PSBa['BarPSperModel'][i])):
            #print(type(PSBa['BarPSperModel'][i][j]['pos_yy']))
            #sys.exit()
            if PSBa['BarPSperModel'][i][j]['pos_yy'].size < 1:
                PSBa['BarPSperModel'][i][j]['utm_pos_lat'] = np.array([])
                PSBa['BarPSperModel'][i][j]['utm_pos_lon'] = np.array([])
                PSBa['BarPSperModel'][i][j]['utm_pos_nr'] = np.array([])
                PSBa['BarPSperModel'][i][j]['utm_pos_reg'] = np.array([])
                pass
            else:
                a = utm.from_latlon(np.array(PSBa['BarPSperModel'][i][j]['pos_yy']), np.array(PSBa['BarPSperModel'][i][j]['pos_xx']), ee['ee_utm'][2])
                PSBa['BarPSperModel'][i][j]['utm_pos_lat'] = a[0]
                PSBa['BarPSperModel'][i][j]['utm_pos_lon'] = a[1]
                PSBa['BarPSperModel'][i][j]['utm_pos_nr']  = a[2]
                PSBa['BarPSperModel'][i][j]['utm_pos_reg'] = a[3]


    return long, PSBa

@jit(nopython=True, cache=True)
def ray_tracing_method(x,y,poly):

    n = len(poly)
    inside = False

    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

def NormMultiDvec(**kwargs):

    """
    # Here mu and sigma, already inserted into ee dictionary
    # Coordinates in utm
    mu = tmpmu =PosMean_3D = [EarlyEst.lonUTM,EarlyEst.latUTM,EarlyEst.Dep*1.E3]
    Sigma = tmpCOV = EarlyEst.PosCovMat_3D = [EarlyEst.PosSigmaXX EarlyEst.PosSigmaXY EarlyEst.PosSigmaXZ; ...
                         EarlyEst.PosSigmaXY EarlyEst.PosSigmaYY EarlyEst.PosSigmaYZ; ...
                         EarlyEst.PosSigmaXZ EarlyEst.PosSigmaYZ EarlyEst.PosSigmaZZ];
    mu =     np.array([ee['lon'], ee['lat'], ee['depth']*1000.0])
    sigma =  np.array([[ee['cov_matrix']['XX'], ee['cov_matrix']['XY'], ee['cov_matrix']['XZ']], \
                       [ee['cov_matrix']['XY'], ee['cov_matrix']['YY'], ee['cov_matrix']['YZ']], \
                       [ee['cov_matrix']['XZ'], ee['cov_matrix']['YZ'], ee['cov_matrix']['ZZ']]])
    """

    x     = kwargs.get('x', None)
    mu    = kwargs.get('mu', None)
    sigma = kwargs.get('sigma', None)

    n = len(mu)

    #mu = np.reshape(mu,(3,1))
    mu = np.reshape(mu,(n,1))
    t1  = (2 * math.pi)**(-1*len(mu)/2)
    t2  = 1 / math.sqrt(np.linalg.det(sigma))
    #c1  = npm.repmat(mu, 1, np.shape(mu)[0])
    c1  = npm.repmat(mu, 1, len(x))
    c11 = (x - c1.transpose()).transpose()
    c12 = x - c1.transpose()

    d  = np.linalg.lstsq(sigma, c11, rcond=None)[0]
    e = np.dot(c12, d)
    f = np.multiply(-0.5,np.diag(e))
    g = np.exp(f)
    h = t1 * t2 * g

    return h

In [8]:
# These functions are copied from the ptf_lambda_bsps_load.py
def load_lambda_BSPS(sigma, ee_d, data_folder):

    d = dict()

    ## Variables for lambda
    d['lambdaBSPS']                                = {}
    d['lambdaBSPS']['hypo_utm']                    = np.array([ee_d['ee_utm'][0], \
                                                               ee_d['ee_utm'][1], \
                                                               ee_d['depth'] ])
    d['lambdaBSPS']['utmzone_hypo']                = ee_d['ee_utm'][2]
    d['lambdaBSPS']['NormCov']                     = ee_d['PosCovMat_3d']
    d['lambdaBSPS']['confid_lev']                  = norm.cdf(sigma) - norm.cdf(-1 * sigma)
    d['lambdaBSPS']['dchi2']                       = distributions.chi2.ppf(d['lambdaBSPS']['confid_lev'], 3)
    d['lambdaBSPS']['SD']                          = math.sqrt(d['lambdaBSPS']['dchi2'])
    d['lambdaBSPS']['mesh']                        = get_meshes(ee_d, data_folder)
    d['lambdaBSPS']['covariance_epicenter_volume'] = get_cov_volume(ee_d['PosCovMat_3d'], d['lambdaBSPS']['SD'])
    d['lambdaBSPS']['npts_mw']                     = get_npts_mw(d['lambdaBSPS']['covariance_epicenter_volume'])
    d['lambdaBSPS']['gaussian_ellipsoid']          = get_gaussian_ellipsoid_3d(ee_d, ee_d['PosCovMat_3d'], d['lambdaBSPS']['SD'], d['lambdaBSPS']['npts_mw'])
    d['lambdaBSPS']                                = get_gaussian_ellipsoid_tetraedons(d['lambdaBSPS'], ee_d)

    return d['lambdaBSPS']

def get_meshes(ee, data_folder):

    path  =  os.path.join(data_folder, 'mesh_files')
    faces = ['HA_mesh_faces_x16.dat', 'CA_mesh_faces_x16.dat', 'Cyprus_mesh_faces_x16.dat']
    nodes = ['HA_mesh_nodes_x16.dat', 'CA_mesh_nodes_x16.dat', 'Cyprus_mesh_nodes_x16.dat']
    names = [ 'HeA', 'CaA', 'CyA']

    mesh_d = dict()

    for i in range(len(faces)):

        f = os.path.join(path, faces[i])
        n = os.path.join(path, nodes[i])

        mesh_name                      = names[i]
        mesh_d[mesh_name]              = dict()
        mesh_d[mesh_name]["faces"]     = np.loadtxt(f)
        mesh_d[mesh_name]["nodes"]     = np.loadtxt(n)
        mesh_d[mesh_name]["nodes_utm"] = utm.from_latlon(mesh_d[mesh_name]["nodes"][:, 1],
                                                         mesh_d[mesh_name]["nodes"][:, 2],
                                                         ee['ee_utm'][2])

    return mesh_d

def get_cov_volume(cov_matrix, std):

    w, v = np.linalg.eig(cov_matrix)
    l_major = std*np.sqrt(w[0]) * 1000.0
    l_inter = std*np.sqrt(w[1]) * 1000.0
    l_minor = std*np.sqrt(w[2]) * 1000.0

    volume = (4./3.) * np.pi * l_major * l_inter * l_minor

    return volume

def get_npts_mw(volume):
    """
    Calculate the number of points to define de ellipsoide.
    Fitting function a*x**b found by F.Romano
    """

    a         = 0.6211
    b         = 0.4358
    n_tetra   = 23382 
    vol_tetra = 3.9990e+15

    npts_mw   = np.ceil(a*(volume*n_tetra/vol_tetra)**b).astype(int)
    npts_mw   = max(10, npts_mw)

    return npts_mw

def get_gaussian_ellipsoid_3d(ee, cov, std, npts): 

    center = [ee['ee_utm'][0], ee['ee_utm'][1], ee['depth']*-1000.0]

    cov = cov*1e6
    w, v = np.linalg.eigh(cov)
    if np.any(w < 0):
        print('Warning: negative eigenvalues')
        w = max(w,0)
    w = std * np.sqrt(w)    #get std of the cov matrix

    volume = (4./3.) * np.pi * w[0] * w[1] * w[2]

    # Make 3x 11x11 arrays
    x, y, z = create_sphere(npts)

    x = np.transpose(x)
    y = np.transpose(y)
    z = np.transpose(z)

    # Flattern 11x11 array
    ap = np.array([np.ravel(x), np.ravel(y), np.ravel(z)])

    bp = np.dot(np.dot(v, np.diag(w)), ap) +  \
         np.transpose(np.tile(center, (np.shape(ap)[1], 1)))

    xp = np.reshape(bp[0, :], np.shape(x))
    yp = np.reshape(bp[1, :], np.shape(y))
    zp = np.reshape(bp[2, :], np.shape(z))

    ellipsoid = {'xp':xp, 'yp':yp, 'zp':zp, 'vol': volume}
    print(" --> Volume of the Gaussian Ellipsoid: {:.8e} [m^3]".format(volume))

    return ellipsoid

def create_sphere(n_points=None, radius=None):
    """
    Create a discrete 3D spheric surface (points)
    Reference to create the shere:
       https://it.mathworks.com/matlabcentral/answers/48240-surface-of-a-equation:
       n = 100;
        r = 1.5;
        theta = (-n:2:n)/n*pi;
        phi = (-n:2:n)'/n*pi/2;
        cosphi = cos(phi); cosphi(1) = 0; cosphi(n+1) = 0;
        sintheta = sin(theta); sintheta(1) = 0; sintheta(n+1) = 0;
        x = r*cosphi*cos(theta);
        y = r*cosphi*sintheta;
        z = r*sin(phi)*ones(1,n+1);
        surf(x,y,z)
        xlabel('X'); ylabel('Y'); zlabel('Z')
    """
    if radius is None:
        radius = 1.0

    if n_points is None:
        n_points = 20

    theta = np.matrix(np.arange(-1*n_points,n_points+1,2) / n_points * np.pi)
    phi   = np.matrix(np.arange(-1*n_points,n_points+1,2) / n_points * np.pi / 2)
    phi   = phi.transpose()

    X = radius*np.matmul(np.cos(phi),np.cos(theta))
    Y = radius*np.matmul(np.cos(phi),np.sin(theta))
    Z = radius*np.matmul(np.sin(phi),np.matrix(np.ones(11)))

    # Set to 0 the very small numbers
    X[0] = 0
    X[-1] = 0
    Y[0] = 0
    Y[-1] = 0
    Y[:,0] = 0
    Y[:,-1] = 0

    return X,Y,Z

def get_gaussian_ellipsoid_tetraedons(el, ee):
    """
    From R. Tonini
    """


    xp = el['gaussian_ellipsoid']['xp']
    yp = el['gaussian_ellipsoid']['yp']
    zp = el['gaussian_ellipsoid']['zp'] #*-1.0

    # array preparation for creating tetrahedrons
    # This is like sss on matptf
    sss = np.vstack([xp.flatten(), yp.flatten(), zp.flatten()]).transpose()
    points_xyz = np.unique(sss, axis=0)
    n_points, tmp = np.shape(points_xyz)

    # lon lat conversion
    points_ll = np.zeros((n_points, 3))
    for i in range(n_points):
        points_ll[i, [1, 0]] = utm.to_latlon(points_xyz[i, 0],
                                             points_xyz[i, 1],
                                             ee['ee_utm'][2], ee['ee_utm'][3])

    points_ll[:, 2] = points_xyz[:, 2]

    # Good but slower (0.0030319690704345703 <=> 0.0013072490692138672)
    # from pyhull.delaunay import DelaunayTri
    # tetrahedrons = np.asarray(DelaunayTri(points_ll).vertices)
    # tetrahedron discretization (based on the points on the surface)
    tessellation = scipy.spatial.Delaunay(points_ll)
    tetrahedrons = tessellation.simplices

    # computing barycenters
    tetra_bar          = {}
    tetra_bar["utm_x"] = np.mean(points_xyz[tetrahedrons, 0], axis=1)
    tetra_bar["utm_y"] = np.mean(points_xyz[tetrahedrons, 1], axis=1)
    tetra_bar["lon"]   = np.mean(points_ll[tetrahedrons, 0], axis=1)
    tetra_bar["lat"]   = np.mean(points_ll[tetrahedrons, 1], axis=1)
    tetra_bar["depth"] = np.mean(points_ll[tetrahedrons, 2], axis=1)

    tetra_xyz = np.column_stack((tetra_bar["utm_x"],
                                 tetra_bar["utm_y"],
                                 tetra_bar["depth"]))

    n_tetra = len(tetra_bar["lon"])
    print(" --> N. Tetra in the Gaussian Ellipsoid: {0}".format(n_tetra))

    # computing tetrahedrons volume
    volume = np.zeros((n_tetra))
    for i in range(n_tetra):
        mm = np.column_stack((points_xyz[tetrahedrons[i, :], :],
                              np.array([1, 1, 1, 1])))
        volume[i] = np.abs(np.linalg.det(mm)/6.)

    volume_tot = np.sum(volume)
    print(" --> Volume of Tetra in the Gaussian Ellipsoid: %.8e [m^3]" % volume_tot)

    Vol_diff_perc = (el['gaussian_ellipsoid']['vol'] - volume_tot) / el['gaussian_ellipsoid']['vol']*100
    print(" --> Volume difference Gaussian <--> Tetra: %.2f [%%]" % Vol_diff_perc)

    el['tetra_bar']                 = tetra_bar
    el['tetrahedrons']              = tetrahedrons
    el['gaussian_ellipsoid_volume'] = volume_tot
    el['volumes_elements']          = volume
    el['tetra_xyz']                 = tetra_xyz

    return el    

In [9]:
# These functions are copied from the ptf_lambda_bsps_sep.py file

def separation_lambda_BSPS(ee, lambda_bsps, LongTerm, mesh):

    moho_ll  = np.column_stack((LongTerm['Discretizations']['BS-2_Position']['Val_x'], LongTerm['Discretizations']['BS-2_Position']['Val_y']))
    tetra_ll = np.column_stack((lambda_bsps['tetra_bar']['lon'], lambda_bsps['tetra_bar']['lat']))

    bar_depth_moho = scipy.interpolate.griddata(moho_ll,
                                                LongTerm['Discretizations']['BS-2_Position']['DepthMoho'],
                                                tetra_ll)

    print(' --> Distance between tetra and slabs:')
    mesh = find_tetra_index_for_ps_and_bs(ee, lambda_bsps, mesh, bar_depth_moho, LongTerm['Discretizations']['BS-2_Position']['grid_moho'])

    lambda_bsps = compute_ps_bs_gaussians_general(ee, lambda_bsps, mesh, bar_depth_moho)

    lambda_bsps = compute_ps_bs_gaussians_single_zone(ee, lambda_bsps, mesh)

    lambda_bsps = update_lambda_bsps_dict(lambda_bsps, LongTerm['Regionalization'])

    return lambda_bsps

def find_tetra_index_for_ps_and_bs(ee, lambda_bsps, mesh, moho, grid_moho):

    buffer = 10000
    tt = np.empty((0,3))

    # here make 1 grid moho in utm
    grid_moho_utm = utm.from_latlon(grid_moho[:,1], grid_moho[:,0], ee['ee_utm'][2])
    grid_moho     = np.column_stack((grid_moho_utm[0].transpose(), grid_moho_utm[1].transpose(), (1000*grid_moho[:,2]).transpose()))

    for keys in mesh:
        # Convert lat lon to utm for the baricenter
        mesh[keys]['bari']['utm'] = utm.from_latlon(mesh[keys]['bari']['lat'], mesh[keys]['bari']['lon'], ee['ee_utm'][2])

        tmp_mesh = np.column_stack((mesh[keys]['bari']['utm'][0].transpose(), \
                                    mesh[keys]['bari']['utm'][1].transpose(), \
                                    mesh[keys]['bari']['depth'].transpose()))
        tt_mesh  = np.concatenate((tt, tmp_mesh))

        mesh[keys]['d_dist'] = find_distances_tetra_mesh(tt_mesh, lambda_bsps['tetra_xyz'], buffer, moho, grid_moho)
        
        print('     --> Min distance from slab %s %10.3f [km]' % (mesh[keys]['name'], mesh[keys]['d_dist']['distance_min_value']/1000))
        print('         --> Nr of PS tetra with dist.  < %4.1f [km] from slab %s : %d  (effective: %d)' % \
              (buffer/1000, mesh[keys]['name'], len(mesh[keys]['d_dist']['idx_less_then_buffer']), len(mesh[keys]['d_dist']['idx_less_then_buffer_effective'])))
        print('         --> Nr of BS tetra with dist. >= %4.1f [km] from slab %s : %d  (effective: %d)' % \
              (buffer/1000, mesh[keys]['name'], len(mesh[keys]['d_dist']['idx_more_then_buffer']), len(mesh[keys]['d_dist']['idx_more_then_buffer_effective'])))

    return mesh

def find_distances_tetra_mesh(mesh, tetra, buffer, moho, g_moho):

    d = dict()

    dist   = np.zeros(len(tetra))
    m_dist = np.zeros(len(tetra))

    for i in range(len(tetra)):
        dist[i]   = np.amin(np.linalg.norm(mesh - tetra[i], axis=1))
        m_dist[i] = np.amin(np.linalg.norm(g_moho - tetra[i], axis=1))

    # Check if all below moho
    d['tetra_in_moho']        = True #default

    # Minimal distance
    d['distances_mesh_tetra'] = dist
    d['distance_min_value']   = np.amin(dist)
    d['distance_min_idx']     = np.argmin(dist)
    d['moho_d_mesh_tetra']    = m_dist
    d['moho_d_min_value']     = np.amin(m_dist)
    d['moho_d_min_idx']       = np.argmin(m_dist)


    # All distances min than buffer
    d['idx_less_then_buffer'] = np.where(dist <= buffer)[0]
    d['idx_more_then_buffer'] = np.where(dist >  buffer)[0]

    ## Questa parte qui general 'errore'!!!!
    # select all indx below the surface for the ones into the slab (PS)
    tmp_tetra = np.take(tetra[:,2],d['idx_less_then_buffer'])
    d['tmp']  = np.where(tmp_tetra <= 0)[0]
    d['idx_less_then_buffer_effective'] = d['idx_less_then_buffer'][d['tmp']]

    # select all indx below the surface for the ones outside the slab (BS)
    tmp_tetra = np.take(tetra[:,2],d['idx_more_then_buffer'])
    d['tmp']  = np.where(tmp_tetra <= 0)[0]
    d['idx_more_then_buffer_effective'] = d['idx_more_then_buffer'][d['tmp']]

    tmp_tetra = np.take(tetra[:,2],d['idx_more_then_buffer_effective'])
    tmp_moho  = np.take(moho, d['idx_more_then_buffer_effective'])
    d['tmp']  = np.where((tmp_moho -1*tmp_tetra/1000) <=0)
    d['idx_more_then_buffer_effective'] = d['idx_more_then_buffer_effective'][d['tmp']]

    # Check if this is in moho
    if(len(d['idx_more_then_buffer_effective']) == 0):
        d['tetra_in_moho'] = False

    return d

def compute_ps_bs_gaussians_general(ee, lambda_bsps, mesh, bar_depth_moho):

    tetra = lambda_bsps['tetra_xyz']
    vol = lambda_bsps['volumes_elements']

    hx             = ee['ee_utm'][0]
    hy             = ee['ee_utm'][1]
    hz             = ee['depth']* (1000.0)
    covariance     = ee['PosCovMat_3dm']
    xyz            = np.array([hx, hy, hz])

    # first merge index
    ps_idx    = []
    bs_idx    = []
    bs_ps_idx = []
    gauss_ps_eff = np.array([])
    gauss_bs_eff = np.array([])

    #distances min
    min_d_mesh = sys.float_info.max
    min_d_moho = sys.float_info.max

    n_tetra, _ = tetra.shape

    for keys in mesh:
        ps_idx.extend((mesh[keys]['d_dist']['idx_less_then_buffer_effective']).tolist())
        if (mesh[keys]['d_dist']['moho_d_min_value'] <= min_d_moho):
            min_d_moho = mesh[keys]['d_dist']['moho_d_min_value']
        if (mesh[keys]['d_dist']['distance_min_value'] <= min_d_mesh):
            min_d_mesh = mesh[keys]['d_dist']['distance_min_value']

    ps_idx = np.array(list(set(ps_idx))).astype(int)
    bs_idx = np.setdiff1d(np.arange(n_tetra,dtype=int),ps_idx)

    tetra_dep = tetra[:,2] / 1000.
    bs_idx_moho = np.where((tetra_dep < 0) & (bar_depth_moho-tetra_dep <= 0))[0]
    bs_idx = np.intersect1d(bs_idx, bs_idx_moho);
    bs_ps_idx = np.concatenate((ps_idx, bs_idx))

    ps_tetra = tetra[ps_idx]
    bs_tetra = tetra[bs_idx]

    bs_ps_tetra = tetra[bs_ps_idx]

    ps_tetra[:,2]    = ps_tetra[:,2]* -1
    bs_tetra[:,2]    = bs_tetra[:,2]* -1
    bs_ps_tetra[:,2] = bs_ps_tetra[:,2]* -1

    gauss_ps_eff = scipy.stats.multivariate_normal.pdf(ps_tetra, xyz, covariance)
    gauss_bs_eff = scipy.stats.multivariate_normal.pdf(bs_tetra, xyz, covariance)
    gauss_bs_ps_eff = scipy.stats.multivariate_normal.pdf(bs_ps_tetra, xyz, covariance)

    sum_bs_ps = np.sum(np.multiply(gauss_bs_ps_eff,vol[bs_ps_idx]))

    lambda_ps = np.sum(np.multiply(gauss_ps_eff,vol[ps_idx])) / sum_bs_ps
    lambda_bs = np.sum(np.multiply(gauss_bs_eff,vol[bs_idx])) / sum_bs_ps

    lambda_bsps['lambda_ps']  = lambda_ps
    lambda_bsps['lambda_bs']  = lambda_bs
    lambda_bsps['gauss_ps']   = gauss_ps_eff
    lambda_bsps['gauss_bs']   = gauss_bs_eff

    print(" --> lambda PS: {:6.4e}  Volume PS: {:10.4e} [m^3]".format(lambda_ps, np.sum(vol[ps_idx])))
    print(" --> lambda BS: {:6.4e}  Volume BS: {:10.4e} [m^3]".format(lambda_bs, np.sum(vol[bs_idx])))
    print(" -->                     Volume BS-PS: {:10.4e} [m^3]".format(np.sum(vol[bs_ps_idx])))

    return lambda_bsps

def compute_ps_bs_gaussians_single_zone(ee, lambda_bsps, mesh):

    vol = lambda_bsps['volumes_elements']
    tetra  = lambda_bsps['tetra_xyz']
    
    hx             = ee['ee_utm'][0]
    hy             = ee['ee_utm'][1]
    hz             = ee['depth']* (1000.0)
    covariance     = ee['PosCovMat_3dm']
    xyz            = np.array([hx, hy, hz])
    lambda_ps_sub  = []

    if (lambda_bsps['lambda_ps'] == 0):

        lambda_bsps['lambda_ps_sub']       = [0,0,0]
        lambda_bsps['lambda_ps_on_ps_tot'] = [0,0,0] # Fixed for lambda_mix == False (PS == 0)
        return lambda_bsps

    for keys in mesh:

        # first merge index
        ps_idx    = []
        bs_idx    = []

        # first Compute general PS-BS
        #for keys in mesh:
        ps_idx.extend((mesh[keys]['d_dist']['idx_less_then_buffer_effective']).tolist())
        bs_idx.extend((mesh[keys]['d_dist']['idx_more_then_buffer_effective']).tolist())
        if(len(ps_idx) == 0):
            lambda_ps = 0.0
            lambda_ps_sub.append(lambda_ps)
            print("     --> Single {} lambda PS: {:6.4e} Volume ps: {:10.4e} [m^3]".format(mesh[keys]['name'], lambda_ps, np.sum(vol[ps_idx])))

        else:
            ps_idx    = set(ps_idx)
            ps_idx    = np.array(list(ps_idx))
            ps_tetra    = tetra[ps_idx]
            ps_tetra[:,2]    = ps_tetra[:,2]* -1
            gauss_ps_eff     = scipy.stats.multivariate_normal.pdf(ps_tetra, xyz, covariance)

            sum_ps    = np.sum(np.multiply(gauss_ps_eff,vol[ps_idx]))
            lambda_ps = (np.sum(np.multiply(gauss_ps_eff,vol[ps_idx])) / sum_ps) * lambda_bsps['lambda_ps']
            lambda_ps_sub.append(lambda_ps)

            print("     --> Single {} lambda PS: {:6.4e} Volume ps: {:10.4e} [m^3]".format(mesh[keys]['name'], lambda_ps, np.sum(vol[ps_idx])))

    lambda_bsps['lambda_ps_sub'] = lambda_ps_sub

    # Define LambdsPs for each reagion on total lambda PS
    lambda_bsps['lambda_ps_on_ps_tot'] =  np.array(lambda_bsps['lambda_ps_sub']) / lambda_bsps['lambda_ps']

    return lambda_bsps

def update_lambda_bsps_dict(lambda_bsps, Regionalization):

    mesh_zones = {'0':'[2,23,43,47,48]', '1':'[9,15,53]', '2':'[26,32,34,35]'}

    regionsPerPS    = np.empty(Regionalization['Npoly'])
    regionsPerPS[:] = np.nan

    for key in mesh_zones:
        l = ast.literal_eval(mesh_zones[key])
        regionsPerPS[l] = int(key)

    lambda_bsps['regionsPerPS'] = regionsPerPS

    return lambda_bsps

In [10]:
# These functions are copied from the ptf_pre_selection.py file

def pre_selection_of_scenarios(sigma, ee, LongTermInfo, PSBarInfo, ellipses):

    pre_selection = dict()
    pre_selection = pre_selection_magnitudes(sigma, ee, pre_selection, LongTermInfo['Discretizations']['BS-1_Magnitude'], LongTermInfo['Discretizations']['PS-1_Magnitude'])

    if (pre_selection['BS_scenarios'] == False and pre_selection['PS_scenarios'] == False):
        print(" --> WARNING: Event magnitude out of the discretization used in PTF. Apply Decision Matrix")
        return False
    
    if (pre_selection['BS_scenarios'] == True):
        pre_selection = pre_selection_BS2_position(pre_selection, LongTermInfo['Discretizations']['BS-2_Position'], ellipses['location_ellipse_2d_BS_inn'], ellipses['location_ellipse_2d_BS_out'])
    else:
        pre_selection['BS2_Position_Selection_inn'] = np.array([])

    if (pre_selection['PS_scenarios'] == True):
        pre_selection = pre_selection_PS2_position(pre_selection, LongTermInfo['Discretizations']['PS-2_PositionArea'], ellipses['location_ellipse_2d_PS_inn'], ellipses['location_ellipse_2d_PS_out'])

        pre_selection = pre_selection_Bar_PS_Model(pre_selection, PSBarInfo['BarPSperModel'], ellipses['location_ellipse_2d_PS_inn'])


    return pre_selection

def pre_selection_magnitudes(sigma, ee, pre_selection, BS_mag, PS_mag):

    val_PS = np.array(PS_mag['Val'])
    ID_PS  = list(PS_mag['ID'])
    val_BS = np.array(BS_mag['Val'])
    ID_BS  = list(BS_mag['ID'])

    # Magnitude range given by sigma
    min_mag = ee['mag'] - ee['MagSigma'] * sigma
    max_mag = ee['mag'] + ee['MagSigma'] * sigma

    # PS
    if(max_mag <= val_PS[0]):
        pre_selection['PS_scenarios'] = False
        sel_PS_Mag_val = np.array([])
        sel_PS_Mag_idx = (np.array([]),)
        sel_PS_Mag_IDs = []

    elif(min_mag >= val_PS[-1]):
        pre_selection['PS_scenarios'] = True
        sel_PS_Mag_val = np.array([val_PS[-1]])
        sel_PS_Mag_idx = (np.where(val_PS[-1]),)
        sel_PS_Mag_IDs = [ID_PS[-1]]

    else:
        pre_selection['PS_scenarios'] = True
        sel_PS_Mag_val = val_PS[(val_PS >= min_mag) & (val_PS <= max_mag)]
        sel_PS_Mag_idx = np.where((val_PS >= min_mag) & (val_PS <= max_mag))
        # To fix if mag uncertainty is too small for val_PS element intervals
        # Find closest magnitude
        if(len(sel_PS_Mag_idx[0]) == 0):
            idx = np.array((np.abs(val_PS-max_mag)).argmin())
            sel_PS_Mag_val = np.array([val_PS[idx]])
            sel_PS_Mag_idx = (np.array[idx],)
            sel_PS_Mag_IDs = list(itemgetter(idx)(ID_PS))
        else:
            sel_PS_Mag_IDs = list(itemgetter(*sel_PS_Mag_idx[0])(ID_PS))

    # BS
    if(max_mag <= val_BS[0]):
        pre_selection['BS_scenarios'] = False
        sel_BS_Mag_val = np.array([])
        sel_BS_Mag_idx = (np.array([]),)
        sel_BS_Mag_IDs = []

    elif(min_mag >= val_BS[-1]):
        # sel_BS_Mag_val = np.array([val_BS[-1]])
        pre_selection['BS_scenarios'] = False
        sel_BS_Mag_val = np.array([])
        sel_BS_Mag_idx = (np.array([]),)
        sel_BS_Mag_IDs = []

    else:
        pre_selection['BS_scenarios'] = True
        sel_BS_Mag_val = val_BS[(val_BS >= min_mag) & (val_BS <= max_mag)]
        sel_BS_Mag_idx = np.where((val_BS >= min_mag) & (val_BS <= max_mag))
        # To fix if mag uncertainty is too small for val_BS element intervals
        # Find closest magnitude
        if(len(sel_BS_Mag_idx[0]) == 0):
            idx = np.array((np.abs(val_BS-max_mag)).argmin())
            sel_BS_Mag_val = np.array([val_BS[idx]])
            sel_BS_Mag_idx = (np.array[idx],)
            sel_BS_Mag_IDs = list(itemgetter(idx)(ID_BS))
        else:
            sel_BS_Mag_IDs = list(itemgetter(*sel_BS_Mag_idx[0])(ID_BS))

    pre_selection['sel_PS_Mag_val'] = sel_PS_Mag_val
    pre_selection['sel_PS_Mag_idx'] = sel_PS_Mag_idx
    pre_selection['sel_PS_Mag_IDs'] = sel_PS_Mag_IDs
    pre_selection['sel_BS_Mag_val'] = sel_BS_Mag_val
    pre_selection['sel_BS_Mag_idx'] = sel_BS_Mag_idx
    pre_selection['sel_BS_Mag_IDs'] = sel_BS_Mag_IDs

    print(" --> BS magnitude values: {}".format(sel_BS_Mag_val))
    print(" --> PS magnitude values: {}".format(sel_PS_Mag_val))

    return pre_selection

def pre_selection_BS2_position(pre_selection, BS2_pos, ellipse_2d_inn, ellipse_2d_out):
    """
    This function uses a ray tracing method decorated with cumba
    """

    # https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python
    points     = zip(BS2_pos['utm_y'], BS2_pos['utm_x'])
    inside_inn = [ray_tracing_method(point[0], point[1], ellipse_2d_inn) for point in points]

    points     = zip(BS2_pos['utm_y'], BS2_pos['utm_x'])
    inside_out = [ray_tracing_method(point[0], point[1], ellipse_2d_out) for point in points]

    # Map common indices
    # bool_array       = np.in1d(np.where(inside_out)[0], np.where(inside_inn)[0])
    bool_array       = np.isin(np.where(inside_out)[0], np.where(inside_inn)[0])
    common_positions = np.where(bool_array)[0]

    # fill dictionary
    pre_selection['BS2_Position_Selection_inn']    = np.where(inside_inn)[0]
    pre_selection['BS2_Position_Selection_out']    = np.where(inside_out)[0]
    pre_selection['BS2_Position_Selection_common'] = np.take(pre_selection['BS2_Position_Selection_out'],common_positions)

    print(" --> BS2_Position inner: {:4d} positions found".format(len(pre_selection['BS2_Position_Selection_inn'])))
    print(" --> BS2_Position outer: {:4d} positions found".format(len(pre_selection['BS2_Position_Selection_out'])))

    # if len(pre_selection['BS2_Position_Selection_inn']) == 0 and len(pre_selection['BS2_Position_Selection_out']) != 0:
    if len(pre_selection['BS2_Position_Selection_inn']) == 0:
        pre_selection['BS_scenarios'] = False
        print('Warning: BS are excluded since no scenarios are found in the inner positions (within sigma_inn)')

    return pre_selection

def pre_selection_PS2_position(pre_selection, PS2_pos, ellipse_2d_inn, ellipse_2d_out):
    """
    This function uses a ray tracing method decorated with cumba
    """

    # https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python
    points     = zip(PS2_pos['utm_y'], PS2_pos['utm_x'])
    inside_inn = [ray_tracing_method(point[0], point[1], ellipse_2d_inn) for point in points]

    points     = zip(PS2_pos['utm_y'], PS2_pos['utm_x'])
    inside_out = [ray_tracing_method(point[0], point[1], ellipse_2d_out) for point in points]

    # Map common indices
    # bool_array       = np.in1d(np.where(inside_out)[0], np.where(inside_inn)[0])
    bool_array       = np.isin(np.where(inside_out)[0], np.where(inside_inn)[0])
    common_positions = np.where(bool_array)[0]

    # fill dictionary
    pre_selection['PS2_Position_Selection_inn']    = np.where(inside_inn)[0]
    pre_selection['PS2_Position_Selection_out']    = np.where(inside_out)[0]
    pre_selection['PS2_Position_Selection_common'] = np.take(pre_selection['PS2_Position_Selection_out'],common_positions)

    print(" --> PS2_Position inner: {:4d} positions found".format(len(pre_selection['PS2_Position_Selection_inn'])))
    print(" --> PS2_Position outer: {:4d} positions found".format(len(pre_selection['PS2_Position_Selection_out'])))

    if len(pre_selection['PS2_Position_Selection_inn']) == 0:
        pre_selection['PS_scenarios'] = False
        print('Warning: PS are excluded since no scenarios are found in the inner positions (within sigma_inn)')

    return pre_selection

def pre_selection_Bar_PS_Model(pre_selection, BarPSperModel, ellipse_2d_inn):
    """
    This function uses a ray tracing method decorated with numba
    """

    Selected_PS_Mag_idx = pre_selection['sel_PS_Mag_idx'][0]

    test_dict = dict()

    for i1 in range(len(Selected_PS_Mag_idx)):
        imag = Selected_PS_Mag_idx[i1]
        for imod in range(len(BarPSperModel[imag])):
            if('utm_pos_lat' in BarPSperModel[imag][imod]):
                if(BarPSperModel[imag][imod]['utm_pos_lat'].size >=2):
                    points     = zip(BarPSperModel[imag][imod]['utm_pos_lon'], BarPSperModel[imag][imod]['utm_pos_lat'])
                    inside_inn = [ray_tracing_method(point[0], point[1], ellipse_2d_inn) for point in points]
                elif(BarPSperModel[imag][imod]['utm_pos_lat'].size ==1 ):
                    inside_inn = ray_tracing_method(BarPSperModel[imag][imod]['utm_pos_lon'][0], BarPSperModel[imag][imod]['utm_pos_lat'][0], ellipse_2d_inn)
                else:
                    pass

                Inside_in_BarPSperModel = {'inside' : np.where(inside_inn)[0]}
                test_dict.setdefault(imag, {})[imod] = Inside_in_BarPSperModel

    pre_selection['Inside_in_BarPSperModel'] = test_dict

    return pre_selection

In [11]:
# These functions are copied from the ptf_short_term.py file

def short_term_probability_distribution(ee, negl_prob, LongTermInfo, PSBarInfo, lambda_bsps, pre_selection):

    short_term = dict()
    short_term['DepProbPoints'] = dict() 
    short_term['DepProbTemps'] = dict()  
    short_term['DepProbScenes'] = dict() 

    x = np.size(pre_selection['sel_BS_Mag_val'])
    y = np.size(pre_selection['BS2_Position_Selection_inn'])
    short_term['DepProbScenesN'] = np.zeros((x,y))

    x = np.size(LongTermInfo['Model_Weights']['PS2_Bar']['Wei'])
    y = np.size(LongTermInfo['Discretizations']['PS-1_Magnitude']['Val'])

    short_term['BarProb'] = [[0 for j in range(y)] for i in range(x)]
    short_term['PS_model_YN'] = np.ones([y,x], dtype = int)

    #COMPUTE INTEGRAL FOR MAGNITUDES
    short_term = compute_distribution_for_magnitudes(LongTermInfo['Discretizations'], short_term, ee)

    # Compute Short term Prob with respect PS and BS (total)
    short_term = find_short_term_prob_for_psbs(short_term, negl_prob, lambda_bsps, len(LongTermInfo['Discretizations']['PS-1_Magnitude']['Val']), pre_selection)

    if(short_term['BS_computed_YN'] == False and short_term['PS_computed_YN'] == False):
        return False

    if(short_term['BS_computed_YN'] == True and pre_selection['BS_scenarios'] == True):
    # COMPUTE INTEGRAL FOR BS POS AND DEPTHS
        short_term = set_grid_integration(LongTermInfo['Discretizations'], pre_selection, short_term)
    # COMPUTE HYPOCENTRAL PROBABILITY DISTRIBUTION FOR BS, IF REQUIRED
        short_term  = get_hypocentral_prob_for_bs(LongTermInfo['Discretizations'], pre_selection, short_term, ee)
    else:
        short_term['Total_BS_Scenarios'] = 0

    # ##COMPUTE PS BAR PROBABILITY DISTRIBUTION FOR EACH PS MODEL, IF REQUIRED
    # if(short_term['PS_computed_YN'] == True and pre_selection['PS_scenarios'] == True):
    #     short_term  = get_ps_bar_probability(Discretizations  = LongTermInfo['Discretizations'],
    #                                          Model_weight     = LongTermInfo['Model_Weights'],
    #                                          pre_selection    = pre_selection,
    #                                          lambda_bsps      = lambda_bsps,
    #                                          PSBarInfo        = PSBarInfo,
    #                                          short_term       = short_term,
    #                                          ee               = ee)
    # else:
    #     short_term['Total_PS_Scenarios'] = 0

    return short_term

def compute_distribution_for_magnitudes(Discretizations, short_term, ee):

    a     = Discretizations['PS-1_Magnitude']['Val'][0:-1]
    b     = Discretizations['PS-1_Magnitude']['Val'][1:]
    c     = np.add(a, b) * 0.5

    lower = np.insert(c, 0, -np.inf)
    upper = np.insert(c, c.size, np.inf)

    lower_probility_norm  = norm.cdf(lower, ee['mag_percentiles']['p50'], ee['MagSigma'])
    upper_probility_norm  = norm.cdf(upper, ee['mag_percentiles']['p50'], ee['MagSigma'])

    short_term['magnitude_probability'] = np.subtract(upper_probility_norm, lower_probility_norm)
    print(' --> Compute magnitude cumulative distribution')

    return short_term

def find_short_term_prob_for_psbs(short_term, negl_prob, lambda_bsps, len_PS_Mag, pre_selection):

    max_BS_mag     = 8.1
    max_PS_mag     = 9.1

    short_term['BS_computed_YN'] = False
    short_term['PS_computed_YN'] = False

    vec_ps = np.ones(len_PS_Mag)  #ones or zeros? should be 1 only for mag>max_mag_BS
    vec_bs = np.zeros(len_PS_Mag)

    if (lambda_bsps['lambda_ps'] != 0.):
        sel_RatioPSonPSTot = np.array(lambda_bsps['lambda_ps_sub']) / lambda_bsps['lambda_ps']
    else:
        sel_RatioPSonPSTot = np.array(lambda_bsps['lambda_ps_sub'])

    pxBS = lambda_bsps['lambda_bs'] / (lambda_bsps['lambda_ps'] + lambda_bsps['lambda_bs'])
    pxPS = 1 - pxBS

    if(pre_selection['BS_scenarios'] == True):
        for i in range(len(pre_selection['sel_BS_Mag_idx'][0])):

            if (pre_selection['sel_BS_Mag_val'][i] <= max_BS_mag):
                vec_bs[pre_selection['sel_BS_Mag_idx'][0][i]] = pxBS

    if(pre_selection['PS_scenarios'] == True):
        for i in range(len(pre_selection['sel_PS_Mag_idx'][0])):

            if (pre_selection['sel_PS_Mag_val'][i] <= max_PS_mag):
                vec_ps[pre_selection['sel_PS_Mag_idx'][0][i]] = pxPS

    short_term['RatioPSonTot'] = vec_ps
    short_term['RatioBSonTot'] = vec_bs
    short_term['sel_RatioPSonPSTot'] = sel_RatioPSonPSTot

    # Check, if probability BS/PS larger than Prob Negligible, then compute PS BS
    tempbs = np.sum(np.multiply(short_term['magnitude_probability'][pre_selection['sel_PS_Mag_idx'][0]], pxBS))
    tempps = np.sum(np.multiply(short_term['magnitude_probability'][pre_selection['sel_PS_Mag_idx'][0]], pxPS))
    if(tempbs > negl_prob):
        short_term['BS_computed_YN'] = True
    if(tempps > negl_prob):
        short_term['PS_computed_YN'] = True

    print(' --> Negligible Probability: %.4f' % negl_prob)
    print(' --> Probability BS = %.4e --> compute BS = %r' % (tempbs, short_term['BS_computed_YN']))
    print(' --> Probability PS = %.4e --> compute PS = %r' % (tempps, short_term['PS_computed_YN']))

    return short_term

def set_grid_integration(Discretizations, pre_selection, short_term):

    z_to_xyfact = 2.5
    space_bin   = 2500.
    space_grid  = z_to_xyfact * space_bin
    all_depth   = np.array([])

    minx = min(Discretizations['BS-2_Position']['utm_x'][pre_selection['BS2_Position_Selection_out']])
    maxx = max(Discretizations['BS-2_Position']['utm_x'][pre_selection['BS2_Position_Selection_out']])
    miny = min(Discretizations['BS-2_Position']['utm_y'][pre_selection['BS2_Position_Selection_out']])
    maxy = max(Discretizations['BS-2_Position']['utm_y'][pre_selection['BS2_Position_Selection_out']])

    tmp = Discretizations['BS-3_Depth']['ValVec'][pre_selection['sel_BS_Mag_idx']]
    for i in range(0, np.shape(tmp)[0]):
        all_depth = np.concatenate((all_depth, tmp[i][pre_selection['BS2_Position_Selection_out']]), axis=None)

    all_depth_bs_3 = np.concatenate(all_depth)*1000
    all_depth_moho = np.array(Discretizations['BS-2_Position']['DepthMoho'])[pre_selection['BS2_Position_Selection_out']]* (-1000.0)

    # check if we have BS positions aligned in x or y or both
    if (minx == maxx):
        x_grid = np.array(minx)
    else:
        x_grid = np.arange(minx, maxx, space_grid)
    if (miny == maxy):
        y_grid = np.array(miny)
    else:
        y_grid = np.arange(miny, maxy, space_grid)

    z_grid = np.arange(min(all_depth_bs_3), max(all_depth_moho), space_bin)

    xx_2d, yy_2d = np.meshgrid(x_grid, y_grid, indexing='xy')
    xx_2d = xx_2d.flatten('F')
    yy_2d = yy_2d.flatten('F')
    grid_2d = np.array([xx_2d, yy_2d])

    xx_3d, yy_3d, zz_3d = np.meshgrid(x_grid, y_grid, z_grid, indexing='xy')
    xx_3d = xx_3d.flatten('F')
    yy_3d = yy_3d.flatten('F')
    zz_3d = zz_3d.flatten('F')
    grid_3d = np.array([xx_3d, yy_3d, zz_3d])

    # inizialize
    dist_2d_idx = np.zeros(len(yy_2d))
    dist_2d_val = np.zeros(len(yy_2d))
    dist_3d_idx = np.zeros(len(yy_3d))
    dist_3d_val = np.zeros(len(yy_3d))

    ## Make array with BS-2_Position
    tmp = [Discretizations['BS-2_Position']['utm_y'][pre_selection['BS2_Position_Selection_out']],
           Discretizations['BS-2_Position']['utm_x'][pre_selection['BS2_Position_Selection_out']]]
    tmp = np.array(tmp).transpose()

    # Mapping: get distances and idx
    for i in range(len(yy_2d)):
        a = np.linalg.norm(tmp - np.array([yy_2d[i], xx_2d[i]]), axis=1)
        idx = np.where(a == np.amin(a))
        dist_2d_idx[i] = idx[0][0]
        dist_2d_val[i] = np.amin(a)

    uu = []
    for i in range(len(yy_3d)):
        a = np.linalg.norm(tmp - np.array([yy_3d[i], xx_3d[i]]), axis=1)
        uu.append(a)
        idx = np.where(a == np.amin(a))
        dist_3d_idx[i] = idx[0][0]
        dist_3d_val[i] = np.amin(a)

    short_term['grid_3d'] = grid_3d
    short_term['grid_2d'] = grid_2d
    short_term['dist_2d_idx'] = dist_2d_idx
    short_term['dist_2d_val'] = dist_2d_val
    short_term['dist_3d_idx'] = dist_3d_idx
    short_term['dist_3d_val'] = dist_3d_val

    print(' --> Set grid Integration')

    return short_term

def get_hypocentral_prob_for_bs(Discretizations, pre_selection, short_term, ee):

    # initialize
    bs1_mag = len(Discretizations['BS-1_Magnitude']['Val'])
    pre_bs  = len(pre_selection['BS2_Position_Selection_inn'])

    tmp_BS_scenarios_val = np.zeros((bs1_mag, pre_bs))
    pre_selection['sel_BS_Mag_idx'] = pre_selection['sel_BS_Mag_idx'][0]

    # SET SPATIAL PROBABILITY TO 1, SINCE ALREADY INCLUDED IN 3D INTEGRATION
    short_term['PosProb'] = np.ones((len(pre_selection['sel_BS_Mag_idx']),
                                    len(pre_selection['BS2_Position_Selection_inn'])))

    for i in range(len(pre_selection['sel_BS_Mag_idx'])):
        #get magnitude
        v_mag   = pre_selection['sel_BS_Mag_val'][i]
        i_mag   = pre_selection['sel_BS_Mag_idx'][i]

        # Compute vertical half_width with respect the magnitude
        v_hwidth = correct_BS_vertical_position(mag = v_mag)
        h_hwidth = correct_BS_horizontal_position(mag = v_mag)

        mu      = ee['PosMean_3d']
        co      = copy.deepcopy(ee['PosCovMat_3dm'])

        # Correct  Covariance matrix
        co[0,0] = co[0,0] + h_hwidth**2
        co[1,1] = co[1,1] + h_hwidth**2
        co[2,2] = co[2,2] + v_hwidth**2

        for j in range(len(pre_selection['BS2_Position_Selection_inn'])):
            #get position
            j_pos_inn_idx = pre_selection['BS2_Position_Selection_inn'][j]
            # COUNT NUMBER OF SCENARIOS TO TREAT
            try:
                a = len(Discretizations['BS-3_Depth']['ValVec'][i_mag][j_pos_inn_idx])
            except:
                a = 0
                raise Exception('!!!! Error in data: no depth defined !!!!')

            b = len(Discretizations['BS-4_FocalMechanism']['ID'])
            tmp_BS_scenarios_val[i, j] = a*b
            j_pos_out = np.where(pre_selection['BS2_Position_Selection_out'] == j_pos_inn_idx)

            # SELECT POINTS ABOVE MOHO IN THIS CELL
            j_sel = np.where((short_term['grid_3d'][2,:]  > v_hwidth) &
                             (short_term['dist_3d_idx']  == j_pos_out[0][0]) &
                             (short_term['grid_3d'][2,:] <= -1000 * Discretizations['BS-2_Position']['DepthMoho'][j_pos_inn_idx] + v_hwidth))

            tmp_depth = 1000 * Discretizations['BS-3_Depth']['ValVec'][i_mag][j_pos_inn_idx] + v_hwidth

            b = np.array([])
            for n in range(len(tmp_depth)):
                a = np.linalg.norm(tmp_depth[n] - short_term['grid_3d'][2,j_sel], axis=0)
                b = np.append(b, a, axis=0)

            n = len(tmp_depth)
            m = int(len(b)/n)
            b = b.reshape(n,m)
            # refDepthSel3D_val = b.min(axis=0)
            refDepthSel3D_idx = np.where(b == np.amin(b, axis=0))[0]

            # COMPUTE PROBABILITY WITHOUT INTEGRAL
            tmp_idx = pre_selection['BS2_Position_Selection_inn'][j]
            a = pre_selection['BS2_Position_Selection_inn']
            x = Discretizations['BS-2_Position']['utm_x'][tmp_idx]
            y = Discretizations['BS-2_Position']['utm_y'][tmp_idx]

            tmp_pt     = npm.repmat([x,y], n,1)
            tmp_pt     = np.append(tmp_pt, tmp_depth.reshape(n,1), axis=1)
            tmp_grid3d = np.array([short_term['grid_3d'][0][j_sel],short_term['grid_3d'][1][j_sel],short_term['grid_3d'][2][j_sel]])

            # compute probability for each depth point (depending on the size of the fault) and scenarios
            short_term_prob_points = NormMultiDvec(x = tmp_pt, mu = mu, sigma = co) # line 536
            short_term_prob_temps  = NormMultiDvec(x = tmp_grid3d.transpose(), mu = mu, sigma = co)

            # Store short_term_prob_points
            short_term['DepProbPoints'][i,j] = copy.deepcopy(short_term_prob_points)
            short_term['DepProbTemps'][i,j]  = copy.deepcopy(short_term_prob_temps)

            selection_sum = np.zeros(len(Discretizations['BS-3_Depth']['ValVec'][i_mag][j_pos_inn_idx]))

            for k in range(len(Discretizations['BS-3_Depth']['ValVec'][i_mag][j_pos_inn_idx])):
                selection = np.where(refDepthSel3D_idx == k)
                selection_sum[k] = np.sum(short_term_prob_temps[selection])

            short_term['DepProbScenes'][i,j] = copy.deepcopy(selection_sum)


        keys = [z for z in short_term['DepProbScenes'] if z[0] == i]
        vals = [short_term['DepProbScenes'][x] for x in keys]
        NormFact = np.sum(np.hstack(vals))

        keys = [z for z in short_term['DepProbPoints'] if z[0] == i]
        vals = [short_term['DepProbPoints'][x] for x in keys]
        NormFactPoints = np.sum(np.hstack(vals))

        for p in range(len(pre_selection['BS2_Position_Selection_inn'])):
            short_term['DepProbScenes'][i,p] = short_term['DepProbScenes'][i,p]/NormFact
            short_term['DepProbPoints'][i,p] = short_term['DepProbPoints'][i,p]/NormFactPoints

    short_term['Total_BS_Scenarios'] = tmp_BS_scenarios_val.sum()

    return short_term

In [12]:
# These functions are copied from the ptf_probability_scenarios.py file

def compute_probability_scenarios(LongTermInfo, pre_selection, short_term, regions):

    probability_scenarios = dict()
    probability_scenarios['nr_ps_scenarios'] = 0
    probability_scenarios['nr_bs_scenarios'] = 0
    
    if (short_term['BS_computed_YN'] == True and pre_selection['BS_scenarios'] == True):
    
        probability_scenarios['par_scenarios_bs']       = np.zeros( (int(short_term['Total_BS_Scenarios']), 11) )
        probability_scenarios['prob_scenarios_bs_fact'] = np.zeros( (int(short_term['Total_BS_Scenarios']), 5) )
        probability_scenarios = bs_probability_scenarios(short_term, pre_selection, regions, probability_scenarios, LongTermInfo['Discretizations'])

        probability_scenarios['relevant_scenarios_bs'] = np.unique(probability_scenarios['par_scenarios_bs'][:,0])

        print(' --> Used regions for BS : {}'.format(probability_scenarios['relevant_scenarios_bs']))

        # Re-Normalize scenarios (to manage events outside nSigma, BS for large events, ...)
        ProbScenBS = probability_scenarios['prob_scenarios_bs_fact'].prod(axis=1)
        TotProbBS_preNorm = np.sum(ProbScenBS)

        # # saving list of bs scenarios parameters
        # if samp_mode == 'None':
        #     save_bs_scenarios_list(par_scenarios_bs = probability_scenarios['par_scenarios_bs'],
        #                            file_bs_list     = file_bs_list)
    else:
        print('No BS scenarios')
        ProbScenBS = np.empty((0))
        TotProbBS_preNorm = 0
        probability_scenarios['par_scenarios_bs'] = np.empty((0, 0))

    # if (short_term['PS_computed_YN'] == True):

    #     probability_scenarios['par_scenarios_ps'] = np.zeros( (int(short_term['Total_PS_Scenarios']), 7) )
    #     probability_scenarios['prob_scenarios_ps_fact'] = np.zeros( (int(short_term['Total_PS_Scenarios']), 5) )

    #     probability_scenarios = ps_probability_scenarios(PSBarInfo        = PSBarInfo,
    #                                                      short_term       = short_term,
    #                                                      pre_selection    = pre_selection,
    #                                                      prob_scenes      = probability_scenarios,
    #                                                      region_ps        = regions['region_listPs'],
    #                                                      Model_Weights    = LongTermInfo['Model_Weights'],
    #                                                      Scenarios_PS     = Scenarios_PS,
    #                                                      ps1_magnitude    = LongTermInfo['Discretizations']['PS-1_Magnitude'],
    #                                                      lambda_bsps      = lambda_bsps)

    #     probability_scenarios['relevant_scenarios_ps'] = np.unique(probability_scenarios['par_scenarios_ps'][:,0])
    #     logger.info(' --> Used regions for PS : {}'.format(probability_scenarios['relevant_scenarios_ps']))
    #     # Re-Normalize scenarios (to manage events outside nSigma, BS for large events, ...)
    #     ProbScenPS = probability_scenarios['prob_scenarios_ps_fact'].prod(axis=1)
    #     TotProbPS_preNorm = np.sum(ProbScenPS)
    # else:
    #     logger.info('No PS scenarios')
    ProbScenPS = np.empty((0))
    TotProbPS_preNorm = 0
    probability_scenarios['par_scenarios_ps'] = np.empty((0, 0))
    #     # # saving empty file
    #     # f_list_ps = open(file_ps_list, 'w')
    #     # f_list_ps.close()

    if (probability_scenarios == False):
        return False

    TotProb_preNorm = TotProbBS_preNorm + TotProbPS_preNorm

    # No scenarios bs or ps possible
    if (TotProb_preNorm == 0):
        return False

    if (TotProb_preNorm < 1.0):
        ProbScenBS = ProbScenBS / TotProb_preNorm
        ProbScenPS = ProbScenPS / TotProb_preNorm
        print(' --> Total BS scenarios probability pre-renormalization: {:.5f}'.format(TotProbBS_preNorm))
        print(' --> Total PS scenarios probability pre-renormalization: {:.5f}'.format(TotProbPS_preNorm))
        print('     --> Total BS and PS probabilty renormalized to 1')

    probability_scenarios['ProbScenBS'] = ProbScenBS
    probability_scenarios['ProbScenPS'] = ProbScenPS

    # # saving probability of scenarios bs and ps
    # if samp_mode == 'None':
    #     save_probability_scenarios(prob_scenarios_bs = probability_scenarios['ProbScenBS'], 
    #                                prob_scenarios_ps = probability_scenarios['ProbScenPS'],
    #                                workflow_dict     = workflow_dict)
 
    # check on the nr scenarios computed into the two section. Should be identical
    check_bs = 'OK'
    check_ps = 'OK'

    print(' --> Total_BS_Scenarios: {:7d}'.format(probability_scenarios['nr_bs_scenarios']))
    print(' --> Total_PS_Scenarios: {:7d}'.format(probability_scenarios['nr_ps_scenarios']))
 
    if (probability_scenarios['nr_bs_scenarios'] != short_term['Total_BS_Scenarios']):
        check_bs = 'WARNING'
        print(' --> Check Nr BS scenarios: {:7d} <--> {} --> {}'.format(probability_scenarios['nr_bs_scenarios'], short_term['Total_BS_Scenarios'], check_bs))
    # if (probability_scenarios['nr_ps_scenarios'] != short_term['Total_PS_Scenarios']):
    #     check_ps = 'WARNING'
    #     print(' --> Check Nr PS scenarios: {:7d} <--> {} --> {}'.format(probability_scenarios['nr_ps_scenarios'], short_term['Total_PS_Scenarios'], check_ps))

    return probability_scenarios

def bs_probability_scenarios(short_term, pre_selection, region_files, prob_scenes, Discretizations):

    empty_scenarios = [37,38,39,41,43,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,82,83,84,85,86,87,88,89,90,92,100,101,102,103,104,105,108,109]

    region_info = dict()
    regions_nr = []

    iScenBS = 0
    sel_mag = len(pre_selection['sel_BS_Mag_idx'])
    bs2_pos = len(pre_selection['BS2_Position_Selection_common'])
    foc_ids = len(Discretizations['BS-4_FocalMechanism']['ID'])

    for i1 in range(sel_mag):
        imag = pre_selection['sel_BS_Mag_idx'][i1]

        for i2 in range(bs2_pos):
            ipos = pre_selection['BS2_Position_Selection_common'][i2]
            ireg = Discretizations['BS-2_Position']['Region'][ipos]
            
            if ireg in empty_scenarios:
                continue

            if (ireg not in regions_nr):
                region_info = load_region_infos(ireg, region_info, region_files)

                regions_nr.append(ireg)

            RegMeanProb_BS4 = region_info[ireg]['BS4_FocMech_MeanProb_valNorm']
            if(RegMeanProb_BS4.size == 0):
                 print(' --> WARNING: region info %d is empty!!!' % (ireg) )

            ipos_reg = np.where(region_info[ireg]['BS4_FocMech_iPosInRegion'] == ipos+1)[1]
            tmpProbAngles = RegMeanProb_BS4[ipos_reg[0]]

            len_depth_valvec = len(Discretizations['BS-3_Depth']['ValVec'][imag][ipos])

            # I3 (depth) AND I4 (angles) ENUMERATE ALL RELEVANT SCENARIOS FOR EACH MAG AND POS (Equivalent to compute_scenarios_prefixes)
            for i3 in range(len_depth_valvec):

                for i4 in range(foc_ids):
                    mag               = Discretizations['BS-1_Magnitude']['Val'][imag]
                    lon, lat          = Discretizations['BS-2_Position']['Val'][ipos].split()
                    depth             = Discretizations['BS-3_Depth']['ValVec'][imag][ipos][i3]
                    strike, dip, rake = Discretizations['BS-4_FocalMechanism']['Val'][i4].split()
                    area              = Discretizations['BS-5_Area']['ValArea'][ireg-1, imag, i4]
                    length            = Discretizations['BS-5_Area']['ValLen'][ireg-1, imag, i4]
                    slip              = Discretizations['BS-6_Slip']['Val'][ireg-1, imag, i4]

                    prob_scenes['par_scenarios_bs'][iScenBS][0]  = int(ireg)
                    prob_scenes['par_scenarios_bs'][iScenBS][1]  = float(mag)
                    prob_scenes['par_scenarios_bs'][iScenBS][2]  = float(lon)
                    prob_scenes['par_scenarios_bs'][iScenBS][3]  = float(lat)
                    prob_scenes['par_scenarios_bs'][iScenBS][4]  = float(depth)
                    prob_scenes['par_scenarios_bs'][iScenBS][5]  = float(strike)
                    prob_scenes['par_scenarios_bs'][iScenBS][6]  = float(dip)
                    prob_scenes['par_scenarios_bs'][iScenBS][7]  = float(rake)
                    prob_scenes['par_scenarios_bs'][iScenBS][8]  = float(length)
                    prob_scenes['par_scenarios_bs'][iScenBS][9]  = float(area)
                    prob_scenes['par_scenarios_bs'][iScenBS][10] = float(slip)

                    prob_scenes['prob_scenarios_bs_fact'][iScenBS][0] = short_term['magnitude_probability'][imag]
                    prob_scenes['prob_scenarios_bs_fact'][iScenBS][1] = short_term['PosProb'][i1, i2]
                    prob_scenes['prob_scenarios_bs_fact'][iScenBS][2] = short_term['RatioBSonTot'][imag]
                    prob_scenes['prob_scenarios_bs_fact'][iScenBS][3] = short_term['DepProbScenes'][i1, i2][i3]
                    prob_scenes['prob_scenarios_bs_fact'][iScenBS][4] = tmpProbAngles[i4]

                    iScenBS = iScenBS + 1

    prob_scenes['par_scenarios_bs'] = prob_scenes['par_scenarios_bs'][:iScenBS,:]
    prob_scenes['prob_scenarios_bs_fact'] = prob_scenes['prob_scenarios_bs_fact'][:iScenBS,:]
    prob_scenes['nr_bs_scenarios'] = np.shape(prob_scenes['prob_scenarios_bs_fact'])[0]

    return prob_scenes

def load_region_infos(ireg, region_info, files):

    info = np.load(files['ModelsProb_Region_files'][ireg-1], allow_pickle=True).item()
    region_info[ireg] = info
    return region_info

In [ ]:
# These functions are copied from the step3_run.py file

def compute_hazard_curves(mih, prob_scenarios, n_pois, thresholds, sigma):

    n_thr = len(thresholds)

    hazard_curves_pois = np.zeros((n_pois, n_thr))

    # print(n_pois, n_thr, n_scen)
    # print(mih.shape, prob_scenarios.shape)

    # hazard_mode = 'lognormal'
    for ip in range(n_pois):

            mih_at_poi = mih[:,ip]
            ind_tmp = np.array(mih_at_poi == 0)
            mih_at_poi[ind_tmp] = 1.e-12

            mu = mih_at_poi
            mu = mu.reshape(len(mu), 1)

            cond_hazard_curve_tmp = 1 - scipy.stats.lognorm.cdf(thresholds, sigma, scale=mu).transpose()
            hazard_curves_pois[ip,:] = np.sum(prob_scenarios*cond_hazard_curve_tmp, axis=1)

    # # hazard_mode = 'lognormal_v1'
    # mih_coo = scipy.sparse.coo_array(mih)
    # print("Number of non-zero elements = {}".format(len(mih_coo.data)))
    # df_mihs = pd.DataFrame({"id_scen":mih_coo.row,"id_poi":mih_coo.col,"mih_value":mih_coo.data})
    # df_prob_scenarios = pd.DataFrame(prob_scenarios)\
    #                             .reset_index()\
    #                             .rename(columns={'index':'id_scen',0:'prob_scen'})
    # df_mihs = df_mihs.merge(df_prob_scenarios,how='left',left_on='id_scen', right_on='id_scen')

    # for ith,threshold in enumerate(thresholds[:]):
    #     df_mihs_thr = df_mihs.copy(deep=True)
    #     col_name = 'prob_lognorm_{}'.format(threshold)
    #     df_mihs_thr[col_name] = 1-scipy.stats.lognorm.cdf(threshold, sigma, scale=df_mihs_thr['mih_value']).transpose()
    #     df_mihs_thr[col_name] = df_mihs_thr[col_name]*df_mihs_thr['prob_scen']
    #     df_mihs_thr = df_mihs_thr.groupby(by='id_poi').agg({col_name:'sum'})
    #     hazard_curves_pois[df_mihs_thr.index,ith]=df_mihs_thr[col_name].to_numpy()

    return hazard_curves_pois

In [ ]:
# These functions are copied from the step5_run.py file

def plot_hazard_maps(points, hmaps, event_dict, map_label):
    """
    """

    proj = cartopy.crs.PlateCarree()
    #cmap = plt.cm.magma_r
    cmap = plt.cm.jet
    ev_lon = event_dict['lon']
    ev_lat = event_dict['lat']
    ev_depth = event_dict['depth']
    ev_mag = event_dict['mag']
    ev_place = event_dict['place']

    for key, hmap in hmaps.items():           
                
        print("mapping ... {}".format(key))
        fig = plt.figure(figsize=(16, 8))
        ax = plt.axes(projection=cartopy.crs.Mercator())
        coastline = cartopy.feature.GSHHSFeature(scale='low', levels=[1])
        #coastline = cartopy.feature.GSHHSFeature(scale='high', levels=[1])
        ax.add_feature(coastline, edgecolor='#000000', facecolor='#cccccc', linewidth=1)
        ax.add_feature(cartopy.feature.BORDERS.with_scale('50m'))
        ax.add_feature(cartopy.feature.STATES.with_scale('50m'))
        ax.add_feature(cartopy.feature.OCEAN.with_scale('50m'))
        gl = ax.gridlines(crs=proj, draw_labels=True, linewidth=1,
                           color="#ffffff", alpha=0.5, linestyle='-')
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = True
        gl.left_labels = True
        gl.xformatter = cartopy.mpl.gridliner.LONGITUDE_FORMATTER
        gl.yformatter = cartopy.mpl.gridliner.LATITUDE_FORMATTER
        gl.xlabel_style = {'size': 14}
        gl.ylabel_style = {'size': 14}

        sc = ax.scatter(points[:,0], points[:,1], c=hmap, s=17, marker="o", 
                    linewidths=0.75, edgecolors="#000000", label=" ",
                    cmap=cmap, clip_on=True,# vmin=map_min, vmax=map_max, 
                    transform=proj, zorder=10, norm=matplotlib.colors.LogNorm(vmin=0.01, vmax=50))#(vmin=map_min, vmax=map_max))

        ax.plot(ev_lon, ev_lat, linewidth=0, marker='*', markersize=14, 
                #markerfacecolor='#c0bfbc', markeredgecolor='#000000', 
                markerfacecolor='white', markeredgecolor='#000000', 
                transform=proj)

        cbar = plt.colorbar(sc, shrink=0.75)
        #cbar.ax.set_yticklabels(labels=cbar.ax.get_yticklabels(), fontsize=10)
        cbar.set_label(label=f'(m)', size=12)
        # ax.set_title("Hazard map - {0}".format(key))
        plt.suptitle("Hazard map - {0}".format(key),fontsize=24)
        plt.title("Epicentral Region: {0} \n Event parameters: Lon={1}, Lat={2}, Depth={3}; Magnitude={4}".format(ev_place, ev_lon, ev_lat, ev_depth, str(ev_mag)[:3] ),fontsize=18)
        ax.set_xlabel(r'Longitude ($^\circ$)', fontsize=14)
        ax.set_ylabel(r'Latitude ($^\circ$)', fontsize=14)
        
def plot_hazard_curve(ind, thresholds, hc):

    fig = plt.figure(figsize=(20,14))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_facecolor('#cccccc')
    ax.grid(True, color='#ffffff')

    for poi_index in ind:

        ax.plot(thresholds, hc[poi_index,:], label=poi_index,
                 alpha=1, # color='#1a5fb4',
                 linestyle="solid", linewidth=3)


    #ax.legend(loc="lower left", ncol=2, bbox_to_anchor=(0., 1.0), frameon=False)7.574120044708252sec
    ax.legend(loc="upper right", ncol=1, bbox_to_anchor=(1.0, 1.0), frameon=True)
    # ax.set_xscale("log")
    ax.set_yscale("log")
    # ax.set_xlim(1e-2, 100)
    ax.set_xlim(0, 20)
    ax.set_ylim(1e-5, 5)
    ax.set_xlabel(r'MIH (m)')
    ax.set_ylabel(r'PoE (50 yrs)')